# **THE TRUTH IS OUT THERE 1°parte** 

Por: [Leonardo Genzano](https://www.linkedin.com/in/leonardo-genzano-1b275193/)

Este notebook se realiza aplicando los conocimientos aprendidos en el curso "Ciencia de datos 360 en Python" de la Escuela de Datos Vivos<br>
https://escueladedatosvivos.ai/courses<br>
Hay cursos muy accesibles y algunos completamente gratuitos

Datos: https://www.kaggle.com/NUFORC/ufo-sightings<br>
Es un registro de avistamientos de OVNI llevado a cabo por National UFO Reporting Center (NUFORC)<br>
Contiene registros de avistamientos desde 1906 hasta 2014<br>
En este notebook haremos una pequeña preparación de datos para dejar un set limpio para analizar luego con visualizaciones. Por ese motivo no va a ser un notebook muy entretenido. Las visualizaciones las hare en otro notebook<br>
2° Parte: https://www.kaggle.com/leogenzano/ufo-sightings-2-parte-analisis-exploratorio/<br>

Consideraciones:
* voy a estar utilizando algunas funciones de la libreria funpymodeling, que nos ayuda sobretodo con el analisis exploratorio y la preparación de datos https://pypi.org/project/funpymodeling/
* Tambien voy a usar la libreria reverse_geocoding para encontrar nombres de países a partir de sus coordenadas https://github.com/thampiman/reverse-geocoder
* en algunos casos donde el código rompe por algun motivo, oculte el output para no hacer largo el notebook. Podes verlo presionando el boton Output arriba a la derecha de cada celda
* Este notebook no fue realizado por un experto. Seguramente haya mejores maneras de afrontar las mismas problematicas. Quedo atento a cualquier comentario o sugerencia 

![The kaggle logo][1]
[1]: https://cdn.forbes.com.mx/2015/03/Internet-XFiles-Mulder.jpg

**Importamos librerias que vamos a utilizar y cargamos los datos en un dataframe**

In [ ]:
pip install funpymodeling

In [ ]:
pip install reverse_geocoder

In [ ]:
import pandas as pd
from funpymodeling.exploratory import freq_tbl, status, profiling_num, cat_vars, num_vars
from pandas_profiling import ProfileReport
import reverse_geocoder as rg

In [ ]:
# Cargamos los datos
data = pd.read_csv("../input/ufo-sightings/scrubbed.csv", sep=",")

# **Análisis inicial**

Variables de los datos
* datetime: fecha y hora del avistamiento
* country: país
* state: estado/provincia/region
* city: ciudad
* shape: forma del ovni
* duration(seconds): duración en segundos del avistamiento
* duration (hours/min): duración en horas y minutos del avistamiento
* comments: descripción 
* datePosted: fecha en que fue cargado el registro
* latitude & longitude: coordenadas geográficas

In [ ]:
status(data)

Con la función status de funpymodeling, verificamos el estado de salud de los datos
* Tipos de datos: excepto por longitude, todos vienen en object.
* Hay muchisimos nulos en country y shape. Esto a pesar de tener registrado por ejemplo el estado o ciudad
* Tambien tenemos nulos en city, state, shape y comments.
* Tenemos dos columnas que muestran el tiempo que duró el avistamiento. Nos quedaremos con la de segundos ya que tiene solo numeros, pero al final la transformaremos a minutos.

In [ ]:
# Veamos algunos datos
data.head()

INDICE<br>
<br>
CONVERSIÓN DE TIPO DE DATOS<br>
* 1° OBSERVACIÓN: 'DATETIME' TIENE COMO TIPO DE DATOS: OBJECT<br>
* 2° OBSERVACIÓN. 'LATITUDE' TIENE COMO TIPO DE DATOS: OBJECT<br>
* 3° OBSERVACIÓN. TENEMOS DOS COLUMNAS PARA EL TIEMPO DE AVISTAMIENTO. AMBAS FIGURAN COMO OBJECT<br>
* 4° OBSERVACIÓN. DATE POSTED FIGURA COMO OBJECT<br>
<br>
TRATAMIENTO DE NULOS<br>
* 5° OBSERVACIÓN: NULOS EN COUNTRY, STATE y CITY<br>
* 6° OBSERVACIÓN: NULOS EN SHAPE Y COMMENTS<br>
<br>
OTRAS TRANSFORMACIONES<br>
* 7° OBSERVACIÓN: CREAR NUEVA COLUMNA DE TIEMPO DE AVISTAMIENTO (MINUTOS)<br>
<br>
ULTIMOS PASOS<br>
* (EDIT) 8° OBSERVACIÓN: ASIGNAR NOMBRE AMIGABLE A LOS PAISES<br>

ALGUNAS ACLARACIONES

* No voy a estar reemplazando las variables originales, les asignare otro nombre y las eliminare al final cuando este seguro de que me quedo como quiero

****************************************************************************************

![The kaggle logo][2]
[2]: https://i.ytimg.com/vi/FtyEo-8OWNA/hqdefault.jpg

# **CONVERSION DE TIPO DE DATOS**

#1° OBSERVACIÓN: 'DATETIME' TIENE COMO TIPO DE DATOS: OBJECT<br>
<br>
Enfoque: convertirlo a datetime64

In [ ]:
data['datetime_dat']=data['datetime'].astype('datetime64')
# No permite hacer la conversion porque aparentemente algunas fechas tienen la hora como '24:00'
# Entonces vamos a reemplazarlo por '00:00'


In [ ]:
# verifico si efectivamente hay registros con la hora cargada como '24:00'
data[data['datetime'].str.contains("24:00")]
# hay 694 regirstros con hora '24:00'

In [ ]:
# donde encuentra un '24:00' le asigna '00:00'
data['datetime_dat'] = data['datetime'].replace({'24:00': '00:00'}, regex=True)

In [ ]:
# Probemos ahora la conversión al tipo de datos datetime
data['datetime_dat']=data['datetime_dat'].astype('datetime64')

**********************

#2° OBSERVACIÓN: 'LATITUDE' TIENE COMO TIPO DE DATOS: OBJECT<br>
<br>
Enfoque: convertirlo a float64

In [ ]:
data['latitude_num']=data['latitude'].astype('float64')
# no lo permite porque encontró un registro donde aparece una letra 'q'. 
# desconozco si es algo propio de la codificacion de las latitudes. Pero como es solo un registro, lo elimino 

In [ ]:
# veamos qué registro es
data[data['latitude'].str.contains('[A-Za-z]', na=False)]  
# es el registro 43782. Vemos que en latitud aparece una 'q' en el medio

In [ ]:
# Vemos que es el 43782. Vamos a eliminarlo con su índice 
data= data.drop([43782])

In [ ]:
# MUY IMPORTANTE
# en este punto, es recomendable reiniciar el índice de la tabla, 
# ya que luego obtendremos los países faltantes a traves de reverse_geocoding.
data = data.reset_index(drop=True)

In [ ]:
# probemos otra vez la conversión de object a numerico
data['latitude_num']=data['latitude'].astype('float64')
# ahora si nos permitió

**********************

#3° OBSERVACIÓN. TENEMOS DOS COLUMNAS PARA EL TIEMPO DE AVISTAMIENTO. AMBAS FIGURAN COMO OBJECT<br>
<br>
Enfoque: eliminar la que muestra en minutos, ya que tiene muchisimos caracteres no numericos (lo haremos al final del notebook)

In [ ]:
# me quedo con la que muestra en segundos (probablemente al final la transformemos a minutos para facilitar el analisis)
# entonces, tomamos 'duration (seconds)' y la transformamos a float64
data['duration(seconds)_num']=data['duration (seconds)'].astype('float64')
# No lo puede converir porque encuentra valores como '2`', es decir tienen caracteres no numericos

In [ ]:
# veamos cuáles son esos registros que traen apostrofe
data[data['duration (seconds)'].str.contains("`",na=False)]
# son 3, podriamos borrarlos. O mejor aun, probemos otra cosa
# vamos a editarle el valor, como hicimos en la 1°observación (reemplazar '24:00' por '00:00')

In [ ]:
# le asigno a cada combinacion de fila/columna, el valor sin apostrofe.
data.loc[27822,'duration (seconds)'] = 2
data.loc[35692,'duration (seconds)'] = 8
data.loc[58590,'duration (seconds)'] = 0.5

In [ ]:
# ahora si, procedemos a convertir el tipo de datos a float64

data['duration(seconds)_num']=data['duration (seconds)'].astype('float64')

**********************

#4° OBSERVACIÓN. DATE POSTED FIGURA COMO OBJECT<br>
<br>
Enfoque: convertirlo a datetime64

In [ ]:
data['datePosted_dat']=data['date posted'].astype('datetime64')
#este no dio problemas. Ya tenemos la conversión

Terminamos por el momento con las conversiones de datos. Proximo paso: tratamiento de nulos

****************************************************************************************

![The kaggle logo][3]
[3]: https://elcomercio.pe/resizer/CMBy5GAqgqFYqvWBHQFqMM58u7A=/580x330/smart/filters:format(jpeg):quality(75)/cloudfront-us-east-1.images.arcpublishing.com/elcomercio/3RKS3OTR4BFUBIXJOKYADRE4AI.jpg

#TRATAMIENTO DE NULOS

Veamos como estan nuestros datos

In [ ]:
status(data)

* recordemos que estamos duplicando las columnas, al final eliminaremos las originales que ya tienen su "arreglo"
* vemos que hay nulos en state, country, shape y comments

#5° OBSERVACIÓN: NULOS EN COUNTRY, STATE y CITY<br>
<br>
Enfoque: obtenerlos a partir de sus coordenadas

Hay 9669 nulos en country. Podriamos eliminarlos...<br>
..Pero si yo tengo correctamente las coordenadas, podría obtener su país a traves de ellas<br>
Esto se logra gracias a reverse_geocoding

In [ ]:
# creo una variable con la tupla de lat y long (para pasarle a reverse_geocoding y haga su magia)
coordinates = list(zip(data['latitude_num'], data['longitude ']))

In [ ]:
import reverse_geocoder as rg

In [ ]:
# le paso al geocoding mis coordenadas para que me devuelva países en una lista
results = rg.search(coordinates) # default mode = 2
# print(results)

Por ejemplo, para el registro 1 nos devuelve:<br>
'lat' : '29.38663':<br>
'lon' : '-98.61797':<br>
'name': 'Lackland Air Force Base':<br>
'admin1': 'Texas':<br>
'admin2': 'Bexar County' (este campo no lo vamos a usar)<br>
'cc': 'US':<br>

Ahora veamos que tiene el primer registro de nuestros datos

In [ ]:
data.loc[1,['latitude','longitude ','city','state','country']]
# los datos coinciden. Este avistamiento fue en Lackland afb(AirForce Base), Texas.
# latitud y longitud vemos que es la misma practicamente (es lo que usa para encontrar países)
# a partir de ahi obtuve su país (aparece como cc), y tambien su estado(admin1) y ciudad(name)

¿Qué hacemos ahora?

In [ ]:
# ahora unimos nuestra data con lo que nos interesa de la lista. Primero convertimos esa lista en un dataframe
results_df = pd.DataFrame(results)

In [ ]:
# ahora simplemente seleccionamos que columnas queremos agregar de results_df a nuestros datos
# ¿se acuerdan que reiniciamos el indice cuando eliminamos un registro en el 2° problema? Fue para hacer esta union facilmente

data['country_c']=results_df['cc']
data['city_c']=results_df['name']
data['state_c'] = results_df['admin1']

In [ ]:
# et voilà, tenemos data de todos esos paises que eran aparecian nulos, a partir de sus coordenadas
(data[['country','country_c','state','state_c','city','city_c']])
# por ejemplo para el registro 1, venia país nulo. Sin embargo ahora sabemos que es de US. Lo confirma que el estado es Texas

Nota: hay algunos estados en esta nueva columna state_c que vienen vacios, a pesar de que los obtuvimos con geocoding<br>
Vamos a reemplazarlos por el mismo valor que aparezca en ciudad

In [ ]:
import numpy as np

In [ ]:
# primero los pasamos a nulo
data['state_c'] = data['state_c'].replace({'': np.nan})

# y ahora a esos nulos le asignamos el nombre de la ciudad
data['state_c'] = data['state_c'].fillna(data['city_c'])

****************

#6° OBSERVACIÓN: NULOS EN SHAPE Y COMMENTS

In [ ]:
# Enfoque: aca si no podemos hacer nada por estos nulos, solo le asignaremos un string 'No data'
data['shape'] = data['shape'].fillna("No data")
data['comments'] = data['comments'].fillna("No data")

***********************************************************************

#OTRAS TRANSFORMACIONES

#7° OBSERVACIÓN: CREAR NUEVA COLUMNA DE TIEMPO DE AVISTAMIENTO (MINUTOS)<br>
Enfoque: Crearemos la columna a partir de la columna de segundos, que tiene datos consistentes

In [ ]:
# 'columna en segundos' / 60 = 'columna en minutos'
data['duration(minutes)_num']=data['duration(seconds)_num'] /60

#ULTIMOS PASOS

Eliminamos todas esas columnas originales que fueron tratadas

In [ ]:
data=data.drop(['datetime','latitude','duration (seconds)','date posted','country','state','city','duration(seconds)_num'],axis=1)
# elimino  tambien duration (hours/min) 
data=data.drop(['duration (hours/min)'],axis=1)

Cambiamos los nombres de las nuevas columnas, y las reordenamos

In [ ]:
data = data.rename(columns={"datetime_dat": "datetime", 
                             "latitude_num": "latitude", 
                             'datePosted_dat' : 'datePosted',
                             'country_c':'country',
                             'state_c':'state',
                             'city_c': 'city',
                             'duration(minutes)_num': 'duration(minutes)',
                             'longitude ': 'longitude' })

data = data[["datetime", "country", "state","city","shape","duration(minutes)","comments", "datePosted","latitude","longitude"]]

Y tenemos nuestro set de datos un poco más ordenado 

In [ ]:
status(data)

*************************

#(EDIT) 8° OBSERVACIÓN: ASIGNAR NOMBRE AMIGABLE A LOS PAISES

Mientras revisaba los ultimos detalles este notebook, se me ocurrio que para visualizar, estaria bueno tener los nombres de los países<br>
En teoría es simple... seria buscar una tabla de países y hacer el join. Veremos que sale

<img src="https://thumbs.gfycat.com/DelightfulDisfiguredHamadryad-size_restricted.gif">

Conseguí esta tabla googleando unos minutos

In [ ]:
countryNames = pd.read_csv('../input/country-code-and-names/Comtrade Country Code and ISO list v1.2.csv', sep =';')

In [ ]:
# Me quedo con las unicas columnas que me van a interesar
countryNames = countryNames.drop(['Country Code','Country Name, Full ','Country Comments','ISO3-digit Alpha','Start Valid Year','End Valid Year'],axis=1)

In [ ]:
# Asi quedó mi tabla de países
countryNames

Hacemos el join (left)

In [ ]:
data_final = data.merge(countryNames, left_on='country', right_on='ISO2-digit Alpha', how='left')

In [ ]:
# Y vemos otra vez el status (...God bless status...)
status(data_final)

Solucionamos los ultimos detalles:

In [ ]:
# todavia hay nulos en Country Name, Abbreviation
# probablemente este sucediendo porque hay codigos de paises en nuestra data principal, que no figuran en nuestra tabla de paises
# entonces, vemos a decirle que si es nulo, muestre el codigo del pais de la tabla principal
data_final['Country Name, Abbreviation'] = data_final['Country Name, Abbreviation'].fillna(data_final['country'])


In [ ]:
# Nota: hay un inconveniente con el país Namibia, Africa
# su codigo es NA, y me parece que se genera una confusion con NAN (nulo)
# para solucionarlo momentaneamente voy a cambiar a mano el valor a "Namibia"
data_final['Country Name, Abbreviation'] = data_final['Country Name, Abbreviation'].replace({'NA': 'Namibia'}, regex=True)

In [ ]:
# Eliminamos las columnas que no nos sirven
data_final = data_final.drop(['country','ISO2-digit Alpha'],axis=1)

In [ ]:
# Renombramos nuestra columna para que quede com 'country'
data_final = data_final.rename(columns={"Country Name, Abbreviation": "country"})

In [ ]:
# Reordenamos
data_final = data_final[["datetime", "country", "state","city","shape","duration(minutes)","comments", "datePosted","latitude","longitude"]]

Y comprobamos una ultima vez.. se ve bien

In [ ]:
status(data_final)

In [ ]:
# Nota: hay punto y coma en el campo de comments. Cuando lo transformo a csv me lo toma como separador de columna
# a pesar de que le declaro que separe solo por comas.
# esto lo soluciona momentaneamente:
data_final['comments'] = data_final['comments'].replace({';': ' '}, regex=True)

Guardamos el set limpio

In [ ]:
data_final.to_csv('clean Data UFO.csv', index=False, sep =',')
#quotechar="'"

<img src="https://i.pinimg.com/originals/a5/42/48/a5424852abc98df69ebd4b4e366e1b6f.gif">

Aca continuaremos el analisis: https://www.kaggle.com/leogenzano/ufo-sightings-2-parte-analisis-exploratorio/